# EGEDA cleaning script

#### For cleaning the October EGEDA energy balance tables excel file provided by ESTO 13 October 2022.
##### This script will create a csv with years across the top.

In [113]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from datetime import datetime
import re

In [114]:
date_today = datetime.now().strftime('%d%m%Y')

In [115]:
# read raw data

RawEGEDA = pd.read_excel('../../data/October_2022/00APEC.xlsx',
                         sheet_name = None, # 'None' reads in all sheets
                         na_values = ['x', 'X', '']) # I don't think there's any x's or X's in the EGEDA xlsx file, but leaving as is (shouldn't make a difference)

In [116]:
economies = RawEGEDA.keys()
economy_dict = pd.read_csv('../../data/economy_dict.csv', header = None, index_col = 0).squeeze('columns').to_dict()

In [117]:
shape_of_data = []

for economy in economies:
    dimension = RawEGEDA[economy].shape
    shape_of_data.append(dimension)

shape_of_data[:5]

[(7844, 44), (7844, 43), (7844, 44), (7844, 44), (7844, 43)]

In [118]:
# Assert whether each data sheet has the same number of rows

def row_check(list):
    return all(i == list[0] for i in list)

number_of_rows = [i[0] for i in shape_of_data]

assert row_check(number_of_rows) == True

In [119]:
# Name the first two columns which are currently blank
for economy in economies:
    RawEGEDA[economy].rename(columns = {'Unnamed: 0': 'product_code', 'Unnamed: 1': 'item_code'}, inplace = True)

In [120]:
# Some regions have 2021 data but it is incomplete, and so we only want data to 2020
years = list(range(1980, 2021, 1))

In [121]:
df_list = []

for economy in economies:
    _df_economy = RawEGEDA[economy]
    _df = pd.melt(_df_economy, 
                  id_vars = ['product_code', 'item_code'], 
                  value_vars = years, 
                  var_name = 'year',
                  value_name = 'value'
                )
    #_df = _df.pivot_table(index=['Year','Product Code'],columns='Item Code',values='Value')
    _df['economy'] = economy_dict[economy]
    _df = _df.set_index(['economy', 'year'])
    df_list.append(_df)

df = pd.concat(df_list)    

In [122]:
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = map(str.lower, df.columns)

# And remove multiple spaces from variables
df['product_code'] = df['product_code'].replace('\s+', ' ', regex = True)
df['item_code'] = df['item_code'].replace('\s+', ' ', regex = True)

In [123]:
# product_code
df['product_code'] = df['product_code'].str.lower()
df['product_code'] = df['product_code'].str.replace(' ', '_', regex = False)\
                                       .str.replace('.', '_', regex = False)\
                                       .str.replace('/', '_', regex = False)\
                                       .str.replace('(', '', regex = False)\
                                       .str.replace(')', '', regex = False)\
                                       .str.replace('-', '', regex = False)\
                                       .str.replace(',', '', regex = False)\
                                       .str.replace('&', 'and', regex = False)\
                                       .str.replace('___', '_', regex = False)\
                                       .str.replace('__', '_', regex = False)\
                                       .str.replace(':', '', regex = False)\
                                       .str.rstrip('_')

# item_code
df['item_code'] = df['item_code'].str.lower()
df['item_code'] = df['item_code'].str.replace(' ', '_', regex = False)\
                                 .str.replace('.', '_', regex = False)\
                                 .str.replace('/', '_', regex = False)\
                                 .str.replace('(', '', regex = False)\
                                 .str.replace(')', '', regex = False)\
                                 .str.replace('-', '', regex = False)\
                                 .str.replace(',', '', regex = False)\
                                 .str.replace('&', 'and', regex = False)\
                                 .str.replace('___', '_', regex = False)\
                                 .str.replace('__', '_', regex = False)\
                                 .str.replace(':', '', regex = False)\
                                 .str.rstrip('_')

In [124]:
# Temporary amendment
df['product_code'] = df['product_code'].str.replace('_0', '_', regex = False)
df['product_code'] = df['product_code'].str.replace('1_refinery_gas', '10_refinery_gas', regex = False)
df['product_code'] = df['product_code'].str.replace('liqour', 'liquor', regex = False)
df['item_code'] = df['item_code'].str.replace('_0', '_', regex = False)

In [125]:
# Just having a quick look at the newly named values in item_code 
list(df.loc[:,'item_code'].unique())[:4]

['1_production', '2_imports', '3_exports', '4_international_marine_bunkers']

In [126]:
df.head(2)

product_code     item_code        value
economy year                                        
01_AUS  1980       1_coal  1_production  2172.907332
        1980       1_coal     2_imports     0.041868

In [127]:
# There are NaN's as per below check
df[df['value'].isna()].head(2)

product_code     item_code  value
economy year                                  
06_HKC  1980       1_coal  1_production    NaN
        1980       1_coal     2_imports    NaN

In [128]:
# Change NaN's to zeroes

df['value'] = df['value'].fillna(0)

In [129]:
# Input thermal coal variable/subtotal

thermal_df = df[df['product_code'].isin(['1_2_other_bituminous_coal', '1_3_subbituminous_coal', '1_4_anthracite', '3_peat', '4_peat_products'])]

assert thermal_df.value.isna().sum() == 0

df1 = thermal_df.groupby(['economy', 'year', 'item_code'])['value'].sum().reset_index().assign(product_code = '1_x_coal_thermal').set_index(['economy', 'year'])

df1 = pd.concat([df, df1])

In [130]:
# And also insert NGL aggregate variable

NGL_df = df1[df1['product_code'].isin(['6_2_natural_gas_liquids', '6_3_refinery_feedstocks', '6_4_additives_oxygenates', '6_5_other_hydrocarbons'])]

assert NGL_df.value.isna().sum() == 0

df2 = NGL_df.groupby(['economy', 'year', 'item_code'])['value'].sum().reset_index().assign(product_code = '6_x_ngls').set_index(['economy', 'year'])

df2 = pd.concat([df1, df2])

In [131]:
# And also insert NGL aggregate variable

otherpet_df = df2[df2['product_code'].isin(['7_12_white_spirit_sbp', '7_13_lubricants', '7_14_bitumen', '7_15_paraffin_waxes', '7_16_petroleum_coke', '7_17_other_products'])]

assert otherpet_df.value.isna().sum() == 0

df3 = otherpet_df.groupby(['economy', 'year', 'item_code'])['value'].sum().reset_index().assign(product_code = '7_x_other_petroleum_products').set_index(['economy', 'year'])

df3 = pd.concat([df2, df3])

In [132]:
# Now jet fuel aggregate variable

jetfuel_df = df3[df3['product_code'].isin(['7_4_gasoline_type_jet_fuel', '7_5_kerosene_type_jet_fuel'])]

assert jetfuel_df.value.isna().sum() == 0

df4 = jetfuel_df.groupby(['economy', 'year', 'item_code'])['value'].sum().reset_index().assign(product_code = '7_x_jet_fuel').set_index(['economy', 'year'])

df4 = pd.concat([df3, df4])

In [133]:
# 16_x_hydrogen fuel (not needed for FED but is needed for TPES)

hydrogen_df = df4[df4['product_code'] == '16_9_other_sources'].copy()

assert hydrogen_df.value.isna().sum() == 0

hydrogen_df.loc[:, 'value'] *= 0

df5 = hydrogen_df.groupby(['economy', 'year', 'item_code'])['value'].sum().reset_index().assign(product_code = '16_x_hydrogen').set_index(['economy', 'year'])

df5 = pd.concat([df4, df5])

In [134]:
df_tidy = df5.reset_index()

In [135]:
# Grabbing new variable category names
# pd.concat([pd.DataFrame(df5.loc[:,'product_code'].unique()),\
#            pd.DataFrame(df5.loc[:,'item_code'].unique())], axis = 1)\
#             .to_csv('../../results/product_item_list_Oct2022.csv', index = False)

In [136]:
# Load correct order of fuel code and item code. Update this csv based on new entries or desired order

ordered = pd.read_csv('../../data/order_Oct2022_egeda.csv')

## Reorder fuel code and item code

In [137]:
# This grabs the unique values of fuel_code and item_code_new in the order they appear in the original dataframe. It removes 'na' by calling '[:-1]' 

order1 = list(ordered['product_code'].unique())[:-1]
order2 = list(ordered['item_code'])

# Take order defined above and define each of the variables as categorical in that already established order (for the benefit of viewing data later)

df_tidy['product_code'] = pd.Categorical(df_tidy['product_code'], 
                                      categories = order1, 
                                      ordered = True)

df_tidy['item_code'] = pd.Categorical(df_tidy['item_code'],
                                          categories = order2,
                                          ordered = True)

In [138]:
df_tidy_sorted = df_tidy.sort_values(['product_code', 'item_code']).reset_index(drop = True)
# df_tidy_sorted[df_tidy_sorted['fuel_code'] == '1_3_subbituminous_coal'] #1_1_3_subbituminous_coal

### Now, pivot the tidy dataset to provide it in wide format similar to RawEGEDA (so years are across the top)

In [139]:
df_years = df_tidy_sorted.pivot_table(index = ['economy', 'product_code', 'item_code'], columns = 'year', values = 'value').reset_index(drop = False)

In [140]:
df_years.to_csv('../../results/EGEDA_2020_created_' + date_today + '.csv', index = False)